In [108]:
import re
from datetime import date,timedelta
import time
import csv

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import random

import nltk
from nltk.tokenize import word_tokenize

from selenium import webdriver
from selenium.webdriver.common.by import By


In [58]:
#PARAMETERS 

POS_FILTER = set(['CC','DT','IN','JJ','JJR','JJS','MD','PDT','PRP','PRP$','RB','RBR','RBS','RP','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WRB'])

MAX_ARTICLES_PER_DAY = 10 
MAX_RETRIES = 3

In [93]:
df = pd.read_csv('article_list.csv')

In [109]:
driver = webdriver.Chrome()
driver.get("https://medium.com")
driver.implicitly_wait(3)
element = driver.find_element(By.XPATH,"//a[text()='Sign in']")
element.click()
driver.implicitly_wait(0.5)
sign_in_with_email = driver.find_element(By.XPATH,'//div[text()="Sign in with email"]')

# Perform actions on the element (e.g., click)
sign_in_with_email.click()
driver.implicitly_wait(0.5)
email = driver.find_element(By.XPATH,"//input[@aria-label='email']")
email.click()
driver.implicitly_wait(0.5)

email.send_keys('hiroakiroa@gmail.com')
driver.implicitly_wait(1)
cont = driver.find_element(By.XPATH,'//button[text()="Continue"]')
cont.click()

In [110]:
signin_url = 'https://medium.com/m/callback/email?token=9fa5097c7dea&operation=login&state=medium&source=email-bcc399e9fcdc-1711121608628-auth.login------0-------------------6c92d325_cf57_4aa5_b307_f162f7fc2e62'
driver.get(signin_url)
driver.implicitly_wait(3)

In [ ]:
days = df[(df['date'].str.contains('2016'))]['date'].unique()
article_df = pd.DataFrame(columns=["url","tag","date","content"])


with open("articles.csv", "w",encoding="utf-8") as file:
    writer = csv.writer(file, dialect='excel')
    writer.writerow(['url','tag','date','content'])
    for day in days:
        print(day, end=" ")
        if df.loc[df['date']==day,:].shape[0] > MAX_ARTICLES_PER_DAY:
          sampled_urls = df.loc[df['date']==day,:].sample(n=MAX_ARTICLES_PER_DAY,replace=False,random_state=0)
        else:
          sampled_urls = df.loc[df['date']==day,:]
        for index, row in sampled_urls.iterrows():
            url= row['url']
            tag= row['tag']
            article_date = row['date']
            content = ""
            retry = 0
            response = None
            try:
                driver.get(url)
                time.sleep(INTERVAL)

            except Exception as e:
                    print("An error occurred:", e)
                    continue
            driver.implicitly_wait(5)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            article = soup.find('article')
            if article is not None:
                for para in article.find_all("p"): 
                    content += para.get_text() + chr(10)
                writer.writerow([url,tag,article_date,content])
file.close()
            
    

2018-03-07 2018-03-08 An error occurred: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00007FF6C51BAD02+56930]
	(No symbol) [0x00007FF6C512F602]
	(No symbol) [0x00007FF6C4FE42E5]
	(No symbol) [0x00007FF6C4FE011A]
	(No symbol) [0x00007FF6C4FD29F4]
	(No symbol) [0x00007FF6C4FD3D29]
	(No symbol) [0x00007FF6C4FD2CF3]
	(No symbol) [0x00007FF6C4FD1EF4]
	(No symbol) [0x00007FF6C4FD1E11]
	(No symbol) [0x00007FF6C4FD0625]
	(No symbol) [0x00007FF6C4FD0EDC]
	(No symbol) [0x00007FF6C4FE69FD]
	(No symbol) [0x00007FF6C5068B47]
	(No symbol) [0x00007FF6C504BC9A]
	(No symbol) [0x00007FF6C50681E2]
	(No symbol) [0x00007FF6C504BA43]
	(No symbol) [0x00007FF6C501D438]
	(No symbol) [0x00007FF6C501E4D1]
	GetHandleVerifier [0x00007FF6C5536F8D+3711213]
	GetHandleVerifier [0x00007FF6C55904CD+4077101]
	GetHandleVerifier [0x00007FF6C558865F+4044735]
	GetHandleVerifier [0x00007FF6C5259736+706710]
	(No symbol) [0x00007FF6C513B8DF]
	(No sy

In [87]:
df = pd.read_csv('articles.csv')
df

,url,tag,date,content,pos content
0,https://blog.devgenius.io/what-the-f-k-chatgpt...,data-science,2023-01-01,Ravi Kumar\r\nFollow\r\nDev Genius\r\n--\r\n3\...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
1,https://blog.devgenius.io/facebook-meta-stock-...,data-science,2023-01-01,Ata Tekeli\r\nFollow\r\nDev Genius\r\n--\r\nLi...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
2,https://medium.com/@chuntcdj/intro-applying-ma...,data-science,2023-01-01,TC. Lin\r\nFollow\r\n--\r\n2\r\nListen\r\nShar...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
3,https://medium.com/@johnvastola/on-the-limitat...,data-science,2023-01-01,Member-only story\r\nJohn Vastola\r\nFollow\r\...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
4,https://medium.com/@zakria.saad3/top-9-vs-code...,data-science,2023-01-01,Zakria Saad\r\nFollow\r\n--\r\nListen\r\nShare...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
5,https://medium.com/@monocosmo77/how-far-have-w...,data-science,2023-01-01,Monodeep Mukherjee\r\nFollow\r\n--\r\nListen\r...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
6,https://medium.com/@rishika-ravindran/what-are...,data-science,2023-01-01,Rishika Ravindran\r\nFollow\r\n--\r\nListen\r\...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
7,https://medium.com/@monocosmo77/how-submodular...,data-science,2023-01-01,Monodeep Mukherjee\r\nFollow\r\n--\r\nListen\r...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
8,https://towardsdev.com/data-engineering-projec...,data-science,2023-01-01,Bar Dadon\r\nFollow\r\nTowards Dev\r\n--\r\n4\...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."
9,https://medium.com/@radovankavicky/news-blog-s...,data-science,2023-01-01,Radovan Kavicky\r\nFollow\r\n--\r\nListen\r\nS...,"[('and', 'CC'), ('a', 'DT'), ('ago', 'RB'), ('..."


In [ ]:
days = df[(df['date'].str.contains('2018'))]['date'].unique()
article_df = pd.DataFrame(columns=["url","tag","date","content"])


with open("articles.csv", "w",encoding="utf-8") as file:
    writer = csv.writer(file, dialect='excel')
    writer.writerow(['url','tag','date','content'])
    for day in days:
        print(day, end=" ")

        sampled_urls = df.loc[df['date']==day,:].sample(n=MAX_ARTICLES_PER_DAY,replace=False,random_state=0)
        for index, row in sampled_urls.iterrows():
            url= row['url']
            tag= row['tag']
            article_date = row['date']
            content = ""
            retry = 0
            response = None
            while (retry < MAX_RETRIES and (response == None or response.status_code != 200)):
                try:
                    response = requests.get(url)
                except Exception as e:
                    print("An error occurred:", e)
                finally:
                    retry += 1
                    time.sleep(INTERVAL)
            if response is not None and response.status_code == 200:
                # Parse the HTML content of the page using BeautifulSoup
                soup = BeautifulSoup(response.content, "html.parser")

                article = soup.find('article')
                for para in article.find_all("p"): 
                    content += para.get_text() + chr(10)
                #row['content'] = content
                #row['pos content'] = filtered_content
                #article_df = pd.concat([article_df,pd.DataFrame(row).transpose()],ignore_index=1,axis=0)
                writer.writerow([url,tag,article_date,content])
            else:
                print("Failed to retrieve the webpage.")

file.close()
            
    

In [107]:
content = ""
retry = 0
response = None
nltk.download('punkt')  # Download the Punkt tokenizer models
nltk.download('averaged_perceptron_tagger')  # Download the POS tagger model

while (retry < MAX_RETRIES and (response == None or response.status_code != 200)):
    response = requests.get("https://medium.com/towards-data-science/its-time-to-say-goodbye-to-pd-read-csv-and-pd-to-csv-27fbc74e84c5")
    retry += 1
    time.sleep(INTERVAL)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    article = soup.find('article')
    for para in article.find_all("p"): 
        content += para.get_text() + chr(10)
else:
    print("Failed to retrieve the webpage.")
print(content)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hiron\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hiron\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Member-only story
Avi Chawla
Follow
Towards Data Science
--
54
Share
Input-output operations with Pandas to a CSV are serialized, making them incredibly inefficient and time-consuming. It's frustrating when I see ample scope for parallelization here, but unfortunately, Pandas does not provide this functionality (yet). Although I am never in favor of creating CSVs in the first place with Pandas (read my post below to know why), I understand that there might be situations where one has no other choice but to work with CSVs.
towardsdatascience.com
Therefore, in this post, we will explore Dask and DataTable, two of the most trending Pandas-like libraries for Data Scientists. We’ll rank Pandas, Dask and Datatable based on their performance on the following parameters:
If we read a CSV through Dask and DataTable, they will generate a Dask DataFrame and DataTable DataFrame respectively, not the Pandas DataFrame. Assuming that we want to stick to the traditional Pandas syntax and…

